# Kneser Toughness Computations

The following notebook is used to assist in the determination of the toughness of Kneser graphs $K(9,4), K(10,4),$ and $K(7,3)$.

In [1]:
import re, subprocess
from math import ceil, comb, floor

We begin with an implementation of the Kruskal-Katona theorem, in order to calculate the lower bound on the neighborhood of a vertex set.

In [2]:
def cascade(n, i):
    a = []
    for k in range(i, 1, -1):
        m = 0
        while comb(m, k) < n:
            m += 1
        a.append(m - 1)
        n -= comb(m - 1, k)
    a.append(n)
    a.append(0)
    a.reverse()
    return a


def kruskal_katona(n, i, r):
    a = cascade(n, i)
    s = 1
    for k in range(i, r, -1):
        s += comb(a[k], k - r)
    return s

We next require a function to generate the partitions of $n$ into $m$ parts in co-lexicographic order with all parts at least some minimum size $\phi$. The following is a modification of Algorithm H from Knuth's *The Art of Computer Programming, Volume 4A*. As a precondition, it requires $n \ge m\phi$.

In [3]:
def partitions(n, m, minimum=1):
    if m == 0:
        if n == 0:
            yield []
    elif m == 1:
        yield [n]
    elif m == 2:
        for i in range(minimum, floor(n / 2) + 1):
            yield [n - i, i]
    else:
        a = [n - (m - 1) * minimum]
        for i in range(m - 1):
            a.append(minimum)
        while True:
            yield a
            while a[1] < a[0] - 1:
                a[0] = a[0] - 1
                a[1] = a[1] + 1
                yield a
            j = 2
            s = a[0] + a[1] - 1
            while j < m and a[j] >= a[0] - 1:
                s += a[j]
                j += 1
            if j == m:
                return
            x = a[j] + 1
            a[j] = x
            j -= 1
            for i in range(j, 0, -1):
                a[i] = x
                s -= x
            a[0] = s

The following function determines the lower bound of $e(S,K(n,k)\setminus S)$ for a given partition $p$ of the $|K(n,k)\setminus S|$ vertices into $c$ components.

In [4]:
def l(f, p):
    s = 0
    for x in p:
        s += f[x]
    return s

The next function searches for the partitions for some $K(n,k)$ and a value of $c$. Let $G$, $d$, and $g$ denote the order, degree, and girth of $K(n,k)$ respectively. Also, let $a$ and $b$ be the number of singletons and $K_2$'s in $K(n,k)\setminus S$. Finally, $u$ is an upper bound on $e(S,K(n,k)\setminus S)$. The algorithm is described in Section 6.2.

In [5]:
def search(n, k, f, c):
    G = comb(n, k)
    d = comb(n - k, k)
    g = min(1 + 2 * ceil(k / (n - 2 * k)), 6 if n == 2 * k + 1 else 4)
    for S in range(c + 1, floor((n / k - 1) * c) + 1):
        u = min(d * S, floor((d + comb(n - k - 1, k - 1)) * S * (comb(n, k) - S) / comb(n, k)))
        a_low = ceil((2 * c * (d - 1) - u) / (d - 2)) # da + (2d - 2)(c - a) <= u, (6.3) with no b term.
        a_high = 1
        while a_high <= c and kruskal_katona(a_high, n - k, n - 2 * k) <= S: # (6.5)
            a_high += 1
        for a in range(a_low, a_high):
            b_low = max(0, ceil((a + g * (c - a) - G + S) / (g - 2)), ceil((d * a + f[g] * (c - a) - u) / (f[g] - 2 * d + 2))) # (6.2), (6.3)
            b_high = 1
            while a + b_high <= c and kruskal_katona(a + b_high, n - k, n - 2 * k) - b_high <= S: # (6.1), (6.6)
                b_high += 1
            for b in range(b_low, b_high):
                for p in partitions(G - S - a - 2 * b, c - a - b, g):
                    if l(f, p) + (2 * d - 2) * b + d * a <= u: # (6.4)
                        print(p + [2] * b + [1] * a)

We'll now focus on $K(9,4)$. From Section 6.1, for $6\leq |\mathcal{C}|\leq 16$, we need to find lower bounds on $e(\mathcal{C},S)$ in order to define $f_3$. Note the cases of $1\leq |\mathcal{C}|\leq 5$ are already covered in the appendix.

In order to use the nauty program, follow the download and installation process found at http://pallini.di.uniroma1.it/. Set up the program such that the executables are in your PATH environment variable. Then the following code will produce lower bounds for $e(\mathcal{C},S)$.

In [6]:
f = [None, 5, 8, None, None, None]
for c in range(6, 17):
    p = subprocess.run('./geng -CtfD5 ' + str(c) + ' | ./countg -g6: --e', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    m = max(int(e) for e in re.findall(r'\d+ graphs : e=(\d+)', p.stdout.decode('utf-8')))
    f.append(5 * c - 2 * m)
for c in range(17, 116):
    f.append(ceil(2 * c * (126 - c) / 126))
print(f)

[None, 5, 8, None, None, None, 18, 21, 22, 25, 26, 27, 28, 29, 28, 31, 32, 30, 31, 33, 34, 35, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 54, 55, 56, 56, 57, 58, 58, 59, 59, 60, 60, 61, 61, 62, 62, 62, 62, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 62, 62, 62, 62, 61, 61, 60, 60, 59, 59, 58, 58, 57, 56, 56, 55, 54, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 39, 38, 37, 35, 34, 33, 31, 30, 28, 27, 25, 24, 22, 21]


Alternatively, if nauty is not installed, initialize $f_3$ with hard coded values.

In [7]:
# f = [None, 5, 8, None, None, None, 18, 21, 22, 25, 26, 27, 28, 29, 28, 31, 32]
# for c in range(17, 117):
#     f.append(ceil(2 * c * (126 - c) / 126))
# print(f)

We can see $f(6)=18\leq f(t)$ for all $6\leq t\leq 116$. Next, checking all possible partitions using the algorithm described in Section 6.2, we find no such partitions, yielding a contradiction.

In [8]:
for c in range(5, 54):
    search(9, 4, f, c)

We'll next focus on $K(10,4)$ and follow the same process. We use nauty to obtain lower bounds on $e(\mathcal{C},S)$.

In [9]:
f = [None, 15, 28, None]
for c in range(4, 9):
    p = subprocess.run('./geng -Ctv ' + str(c), shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    m = max(int(e) for e in re.findall(r'>C \d+ graphs with (\d+) edges', p.stderr.decode('utf-8')))
    f.append(15 * c - 2 * m)
for c in range(9, 95):
    f.append(ceil(9 * c * (210 - c) / 210))
print(f)

[None, 15, 28, None, 52, 63, 72, 81, 88, 78, 86, 94, 102, 110, 118, 126, 134, 141, 149, 156, 163, 171, 178, 185, 192, 199, 206, 212, 219, 225, 232, 238, 245, 251, 257, 263, 269, 275, 281, 286, 292, 297, 303, 308, 314, 319, 324, 329, 334, 339, 343, 348, 353, 357, 362, 366, 370, 374, 378, 382, 386, 390, 394, 397, 401, 404, 408, 411, 414, 417, 420, 423, 426, 429, 432, 434, 437, 439, 442, 444, 446, 448, 450, 452, 454, 456, 458, 459, 461, 462, 463, 465, 466, 467, 468]


Alternatively, the hard coded values of $f_2$ are as follows.

In [ ]:
# f = [None, 15, 28, None, 52, 63, 72, 81, 88]
# for c in range(9, 115):
#     f.append(ceil(9 * c * (210 - c) / 210))
# print(f)

Once again, $f(4)\leq f(t)$ for all $4\leq t\leq 114$. The partition algorithm from Section 6.2 finds no valid partitions, yielding a contradiction.

In [10]:
for c in range(48, 76):
    search(10, 4, f, c)

Once again, $f(4)\leq f(t)$ for all $4\leq t\leq 114$. The partition algorithm from Section 6.2 finds no valid partitions, yielding a contradiction.

In [11]:
for c in range(48, 76):
    search(10, 4, f, c)

Finally, we'll focus on $K(7,3)$. The values of $f_1$ are below.

In [12]:
f = [None, 4, 6, None, None, None, 12, 14]
for c in range(8, 26):
    f.append(ceil(2 * c * (35 - c) / 35))
print(f)

[None, 4, 6, None, None, None, 12, 14, 13, 14, 15, 16, 16, 17, 17, 18, 18, 18, 18, 18, 18, 17, 17, 16, 16, 15]


Again, $f(6)\leq f(t)$ for all $6\leq t\leq 25$. There are no valid partitions, yielding a contradiction.

In [13]:
for c in range(5, 14):
    search(7, 3, f, c)